In [16]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [17]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/FYP")
! pip install hottbox

In [0]:
import numpy as np
#import scipy as sc
import copy
import time
from hottbox.core import Tensor

class LSSVM:
  
  def __init__(self, C = 1, max_iter = 10):
    self.C = 1
    self.max_iter = max_iter
    self.w = None
    self.b = 0
    self.iter = 0
    
    self.b_history = []
    self.orig_label = []
    
  def fit(self, Xtrain, labels):
    
    self.orig_label = list(set(labels))
    binary_labels = [1 if x == self.orig_label[0] else -1 for x in labels]
    for i in range(self.max_iter):
      print(i)
      alphas, b = self.optimizer(Xtrain, binary_labels, self.C)
      w = np.dot(alphas.transpose(), Xtrain).squeeze()
      w = w / np.linalg.norm(w)
      
      self.w = w
      self.b = b
      self.iter = i
      self.b_history.append(b)
      if self.converged(): break
        
  def predict(self, Xtest):
    N = Xtest.shape[0]
    w = self.w
    b = self.b
    predict = []
    dist = []
    
    for im in range(N):
        temp = Xtest[im,:]
        temp = np.dot(w.T, temp)
        dist.append(temp.squeeze()+ b)
        predict.append(np.sign(temp.squeeze() + b))
    predict = [self.orig_label[0] if x == 1 else self.orig_label[1] for x in predict]
    return predict, dist 
    
    
  def optimizer(self, Xtrain, binary_labels, C):
    gamma = 2*C
    M = Xtrain.shape[0]
    y_train = np.expand_dims(binary_labels, axis = 1)
    
    omega = np.dot(Xtrain, Xtrain.T)
    
    left_column = np.append(np.array([0]), np.ones(M))
    left_column = left_column.reshape(M+1, 1)
    right_block = np.append(np.expand_dims(np.ones(M), axis=0),  omega + (1/gamma) * np.eye(M), axis=0)
    LFS = np.append( left_column, right_block, axis=1)
    RHS = np.append(np.array([[0]]), y_train, axis=0)
    
    result = np.dot(np.linalg.inv(LFS), RHS)
    
    b = result[0][0]
    alphas = result[1:, :]
    
    return alphas, b
  
  def converged(self):
    if len(self.b_history) > 5:
      err = np.diff(np.array(self.b_history[-6:]))
      
      if np.all(np.abs(err) < 1e-8):
        return True
      
    return False
    
    

In [51]:
#The whole code works for python 3.
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt

def load_file(filename):
  with open (filename,'rb') as fo:
    data = pickle.load(fo, encoding = 'latin1')
    label = data['labels']
    images = data['data']
    Label = np.array(label)
    Images = images.reshape(10000,3,32,32).transpose(0,2,3,1).astype('float')
  return Label, Images

def load_cifar10(root):
  label_temp = []
  image_temp = []
  for i in range (1,6):
    trainfile = os.path.join(root, 'data_batch_%d' %(i,))
    l,i = load_file(trainfile)
    label_temp.append(l)
    image_temp.append(i)
  label_train = np.concatenate(label_temp)
  image_train = np.concatenate(image_temp)
  testfile = os.path.join(root, 'test_batch')
  label_test, image_test = load_file(testfile)
  
  return label_train, label_test, image_train, image_test

def showimage(label, image):
  classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
  num_class = len(classes)
  num_sample = 7
  for order, class_name in enumerate(classes):
    index = np.flatnonzero(label == order)
    idxs = np.random.choice(index, num_sample, replace = False, p = None)
    for i, idx in enumerate(idxs):
      plt_idx = i * num_class + order + 1
      plt.subplot(num_sample, num_class, plt_idx)
      plt.imshow(image[idx].astype('uint8'))
      plt.axis('off')
      if i == 0:
        plt.title(class_name)
  plt.show()
  
def class_extract(images, label, c, num):
    idx1 = np.flatnonzero(label == c[0])
    idx1_new = np.random.choice(idx1, num, replace = False, p = None)
    idx2 = np.flatnonzero(label == c[1])
    idx2_new = np.random.choice(idx2, num, replace = False, p = None)
    num = idx1_new.shape[0] + idx2_new.shape[0]
    idx = np.hstack((idx1_new,idx2_new))
    idxs = np.random.choice(idx, num, replace = False, p = None)
    class_image = images[idxs]
    class_label = label[idxs]
    
    return class_image, class_label

label_train, label_test, image_train, image_test = load_cifar10('.')
train_im, train_lb = class_extract(image_train, label_train, [0,1], 2000)
avg_im = np.sum(train_im, axis = 0)/train_im.shape[0]
test_im, test_lb = class_extract(image_test, label_test, [0,1], 1000)
train_im = train_im - avg_im
test_im = test_im - avg_im
print(test_im.shape)
print(train_lb.shape)
Image_train = train_im.reshape(4000, -1)
Label_train = train_lb
Image_test = test_im.reshape(2000, -1)

(2000, 32, 32, 3)
(4000,)


In [37]:
print(Image_train[1,2])

-0.8584999999999923


In [57]:
svm = LSSVM(C=10, max_iter=20)
svm.fit(Image_train, Label_train)

0
1
2
3
4
5


In [49]:
a = Image_test[1,:]
print(Image_test.shape)

(4000, 1536)


In [60]:
y_tmp, dd = svm.predict(Image_test)

result = np.zeros(Image_test.shape[0])
succ = 0
for i in range(Image_test.shape[0]):
    
    if y_tmp[i] == test_lb[i]:
       succ = succ + 1
rate = succ/2000
print(rate)

0.6275
